# Qiskit Scratch Work
This notebook contains scratch work from learning how to IMB's quantum Qiskit package.

In [6]:
from dotenv import load_dotenv
from qiskit import QuantumCircuit, transpile
from qiskit.primitives import StatevectorEstimator, StatevectorSampler
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_runtime import QiskitRuntimeService
import numpy as np
import os
import qiskit

load_dotenv()
API_TOKEN = os.getenv("API_TOKEN")

## Install Qiskit
This section follows the installation section from the [IBM Quantum Documentation](https://docs.quantum.ibm.com/guides/install-qiskit).

In [2]:
qiskit.__version__

'1.3.1'

In [3]:
service = QiskitRuntimeService(channel="ibm_quantum", token=API_TOKEN)

"""AccountAlreadyExistsError: 'Named account (default-ibm-quantum) already exists. Set overwrite=True to overwrite.'"""
# QiskitRuntimeService.save_account(channel="ibm_quantum", token=API_TOKEN) 

backend = service.backend(name="ibm_brisbane")

In [4]:
backend.num_qubits

127

## Create your first quantum program in Qiskit
This section follows the `README.md` from the [GitHub repo](https://github.com/Qiskit/qiskit).

In [5]:
# 1. A quantum circuit for preparing the quantum state |000> + i |111>
qc_example = QuantumCircuit(3)
qc_example.h(0)          # generate superpostion
qc_example.p(np.pi/2,0)  # add quantum phase
qc_example.cx(0,1)       # 0th-qubit-Controlled-NOT gate on 1st qubit
qc_example.cx(0,2)       # 0th-qubit-Controlled-NOT gate on 2nd qubit

In [9]:
# 2. Add the classical output in the form of measurement of all qubits
qc_measured = qc_example.measure_all(inplace=False)

# 3. Execute using the Sampler primitive
sampler = StatevectorSampler()
job = sampler.run([qc_measured], shots=1000)
result = job.result()
print(f" > Counts: {result[0].data['meas'].get_counts()}")

 > Counts: {'000': 515, '111': 485}


In [10]:
# 2. Define the observable to be measured 
operator = SparsePauliOp.from_list([("XXY", 1), ("XYX", 1), ("YXX", 1), ("YYY", -1)])

# 3. Execute using the Estimator primitive
estimator = StatevectorEstimator()
job = estimator.run([(qc_example, operator)], precision=1e-3)
result = job.result()
print(f" > Expectation values: {result[0].data.evs}")

 > Expectation values: 3.9994273994307683


In [11]:
qc_transpiled = transpile(qc_example, basis_gates = ['cz', 'sx', 'rz'], coupling_map =[[0, 1], [1, 2]] , optimization_level=3)